# Denison CS181/DA210 Homework 4.b - Step 2

Before you turn this problem in, make sure everything runs as expected. This is a combination of **restarting the kernel** and then **running all cells** (in the menubar, select Kernel$\rightarrow$Restart And Run All).

Make sure you fill in any place that says `# YOUR CODE HERE` or "YOUR ANSWER HERE".

---

#### Import Python modules and load "SQL Magic"

In [1]:
import pandas as pd
import os
import os.path
import json
import sys
import importlib

module_dir = "../../modules"
module_path = os.path.abspath(module_dir)
if not module_path in sys.path:
    sys.path.append(module_path)

%load_ext sql

In [2]:
def getsqlite_creds(dirname=".",filename="creds.json",source="sqlite"):
    """ Using directory and filename parameters, open a credentials file
        and obtain the two parts needed for a connection string to
        a local provider using the "sqlite" dictionary within
        an outer dictionary.  
        
        Return a scheme and a dbfile
    """
    assert os.path.isfile(os.path.join(dirname, filename))
    with open(os.path.join(dirname, filename)) as f:
        D = json.load(f)
    sqlite = D[source]
    return sqlite["scheme"], sqlite["dbdir"], sqlite["database"]

In [3]:
scheme, dbdir, database = getsqlite_creds(source="sqlite1")
template = '{}:///{}/{}.db'
cstring = template.format(scheme, dbdir, database)
print("Connection string:", cstring)

Connection string: sqlite:///../../dbfiles/book.db


#### Establish Connection from Client to Server

In [4]:
%sql $cstring

---

## Part B: SQL Group By Operations

In the following cells, your only action is to remove the `# YOUR CODE HERE` and `raise NotImplementedError()` lines, and then put a valid SQL statement as the **value** of string variable `query`.

In each case, when you execute the cell, the query will be sent to the database management system, a result obtained, and the result converted into a `pandas` data frame, whose prefix is shown.  (This allows for testing the results as well as displaying them in your Jupyter Notebook.)

_Note: you may want to view the `book` database in SQLiteStudio to get an idea of what the field names are as you work through these exercises._

**Q1:** Using the SQL table `countries` in the `book` database, use a select query to answer the question: how many countries are there in each region?  Alias your new column as `new`.

In [9]:
query = """SELECT COUNT(country) AS new
FROM countries
GROUP BY region
"""


resultset1 = %sql $query
resultdf1 = resultset1.DataFrame()
resultdf1

 * sqlite:///../../dbfiles/book.db
Done.


,new
0,37
1,58
2,42
3,21
4,3
5,8
6,48


In [10]:
# Testing cell
assert len(resultdf1) == 7
assert 58 in list(resultdf1['new'])

**Q2:** Use the `indicators` table in the `book` database to find the total world population in each year (as the sum of country populations). Use the alias `total_pop` for your new column.  Sort the result in ascending year value.

In [11]:
query = """SELECT SUM(pop) AS total_pop
FROM indicators
GROUP BY year
ORDER BY total_pop ASC
"""


resultset2 = %sql $query
resultdf2 = resultset2.DataFrame()
resultdf2.head()

 * sqlite:///../../dbfiles/book.db
Done.


,total_pop
0,3014.69
1,3055.37
2,3107.86
3,3172.67
4,3237.70


In [12]:
# Testing cell
assert len(resultdf2) == 59
assert resultdf2.loc[58,'total_pop'] > 7500
assert resultdf2.loc[58,'total_pop'] < 7600

**Q3:** Treating your query above as a subquery (without the ORDER BY), find the minimum for `total_pop` over all years. Use the alias `m` for the new column.

In [16]:
query = """SELECT MIN(total_pop) AS m FROM
(SELECT SUM(pop) AS total_pop
FROM indicators
GROUP BY year)
"""


resultset3 = %sql $query
resultdf3 = resultset3.DataFrame()
resultdf3.head()

 * sqlite:///../../dbfiles/book.db
Done.


,m
0,3014.69


In [17]:
# Testing cell
assert len(resultdf3) == 1
assert resultdf3.loc[0,'m'] > 3014
assert resultdf3.loc[0,'m'] < 3015

**Q4:** Not all countries are growing, so the largest population a country ever had might be in a previous year. For each country code in `indicators`, find the max population that country ever had. Alias your new column as `max_pop`. You should have one row per country. Don't change the order (that is, your records should still be ordered by `code`).

In [20]:
query = """SELECT MAX(pop) AS max_pop FROM indicators
GROUP BY code
"""


resultset4 = %sql $query
resultdf4 = resultset4.DataFrame()
resultdf4.head()

 * sqlite:///../../dbfiles/book.db
Done.


,max_pop
0,0.11
1,37.17
2,30.81
3,3.29
4,0.08


In [21]:
# Testing cell
assert len(resultdf4) == 218
assert resultdf4.loc[0,'max_pop'] == 0.11
assert resultdf4.loc[1,'max_pop'] == 37.17

**Q5:** With reference to the above, find all records where the max population is less than 1 (remember, this is measured in millions of people).  Use a `HAVING` clause.  Keep the original ordering of the data (alphabetically, by `code`).

In [22]:
query = """SELECT MAX(pop) AS max_pop FROM indicators
GROUP BY code
HAVING max_pop < 1
"""


resultset5 = %sql $query
resultdf5 = resultset5.DataFrame()
resultdf5.head()

 * sqlite:///../../dbfiles/book.db
Done.


,max_pop
0,0.11
1,0.08
2,0.06
3,0.10
4,0.39


In [23]:
# Testing cell
assert len(resultdf5) == 58
assert resultdf5.loc[0,'max_pop'] == 0.11
assert resultdf5.loc[1,'max_pop'] == 0.08

**Q6:** Using the `indicators` table in the `book` database, find the total world population in each year (as the sum of country populations), then return the rows where the total population is greater than 6000 (measured in millions of people).  Use the alias `total_pop` for your new column.

In [24]:
query = """SELECT SUM(pop) AS total_pop
FROM indicators
GROUP BY year
HAVING total_pop > 6000
"""


resultset6 = %sql $query
resultdf6 = resultset6.DataFrame()
resultdf6.head()

 * sqlite:///../../dbfiles/book.db
Done.


,total_pop
0,6013.26
1,6092.92
2,6172.14
3,6251.05
4,6330.08


In [25]:
# Testing cell
assert len(resultdf6) == 20
assert resultdf6.loc[0,'total_pop'] < 6014
assert resultdf6.loc[1,'total_pop'] > 6090

---

---

## Part C

**Q7:** How much time (in minutes/hours) did you spend on this homework assignment?

1 hour

**Q8:** Who was your partner for this assignment?  If you worked alone, say so instead.

I worked alone